# Preparation  
## Configuring AWS environment  
https://hands-on.cloud/introduction-to-boto3-library/#configuring-aws-environment  

In [1]:
import os
import networkx as nx
import boto3
import paramiko

# Run arguments
host_os = 'linux_mac' # or 'windows'
data_dir = '/home/rony/services/milestones_chains/data/'
data_file_name = 'EMS_DCMA_DD_23.08.graphml'
data_path = os.path.join(data_dir, data_file_name)
experiment = 'EMS_DCMA_DD_23_08'
tasks = 'tdas'
results = 'prt'

## AWS
# IPs
resultsIP = '172.31.10.240'
serviceIP = '172.31.15.123'
# Compute instance
key_file_name = 'ds_eu_west2_2.pem'
if host_os=='windows':
    key_file_name = 'C:\\Users\\username\\.aws\\ds_eu_west2_2.pem'
else:
    key_file_name = '~/.aws/ds_eu_west2_2.pem'
INSTANCE_ID = 'i-0586e11281d4b02a2'
AWS_REGION = "eu-west-2"
EC2_RESOURCE = boto3.resource('ec2', region_name=AWS_REGION)
EC2_CLIENT = boto3.client('ec2', region_name=AWS_REGION)

# Storage bucket
profile_name = 'ds_sandbox'
data_bucket = 'programsdatabucket'
results_bucket = 'chainsresults'
session = boto3.session.Session(profile_name=profile_name)
S3_CLIENT = session.client('s3')
S3_RESOURCE = boto3.resource('s3')

In [2]:
# Upload data to an S3 bucket
S3_CLIENT.upload_file(data_path, data_bucket, data_file_name)

In [3]:
# Start compute instance
instance = EC2_RESOURCE.Instance(INSTANCE_ID)
instance.start()
print(f'Starting EC2 instance: {instance.id}')
instance.wait_until_running()
print(f'EC2 instance "{instance.id}" has been started')

Starting EC2 instance: i-0586e11281d4b02a2
EC2 instance "i-0586e11281d4b02a2" has been started


In [5]:
# Connect to instance to run process 
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
privkey = paramiko.RSAKey.from_private_key_file('ds_eu_west2_2.pem') # Works only with local pem file
ssh.connect(hostname=serviceIP, username='ubuntu', pkey=privkey)

In [12]:
process_statement = "python3 remote_run_test.py"
process_statement = 'cd services/milestones_chains && python3 service.py {f} {e} {t} {r}'\
.format(f=data_file_name, e=experiment, t=tasks, r=results)
stdin, stdout, stderr = ssh.exec_command(process_statement)
print('stdout:', stdout.readlines())

stdout: ['start\n', 'service started on 12:03:56\n', "args: Namespace(data_file_name='EMS_DCMA_DD_23.08.graphml', experiment='EMS_DCMA_DD_23_08', prt='prt', tasks_types='tdas')\n", 'file EMS_DCMA_DD_23.08.graphml downloaded\n', 'Graph with 4217 nodes and 5092 edges\n', 'CREATE TABLE IF NOT EXISTS MCdb.EMS_DCMA_DD_23_08_chains (id TEXT,chain TEXT);\n', '181 root_successors will be used to build sub-graphs\n', '##################################################\n', '110.7 DiGraph with 3699 nodes and 4723 edges\n', '##################################################\n', '110.9 DiGraph with 3699 nodes and 4723 edges\n', '##################################################\n', '112.5 DiGraph with 3699 nodes and 4723 edges\n', '##################################################\n', '122.7 DiGraph with 3699 nodes and 4723 edges\n', '##################################################\n', '125.7 DiGraph with 3699 nodes and 4723 edges\n', '##################################################\n', '1

In [ ]:
print('stdout:', stdout.read())

In [ ]:
if stderr:
    print('Run attempt encountered error:\n', stderr.readlines())
elif stdout:
    print(stdout.readlines())
    # stdout = stdout.readlines()[0]
    # stdout = stdout.replace('\n', '')
    if stdout == 'completed':
        print(stdout)
        # Stop instance 
        response = EC2_CLIENT.stop_instances(InstanceIds=[INSTANCE_ID], DryRun=False)
        # Delete data from S3 bucket
        s3_resource.Object(data_bucket, data_file_name).delete()

In [ ]:
response = EC2_CLIENT.stop_instances(InstanceIds=[INSTANCE_ID], DryRun=False)